#  Microstructural brain correlates of inter-individual differences in respiratory interoception 

## Notebook for analysis of behavioural data 

### Set up environment

In [1]:
# Use neurosci environment
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import norm
import math

data_dir = '/Users/au657961/github/Respiroception_VBQ/data'
figures_dir = '/Users/au657961/github/Respiroception_VBQ/figures'

Matplotlib created a temporary cache directory at /var/folders/wh/fsqw487n2kz8s4zncj30yj2dk1v6zh/T/matplotlib-7ddlhm09 because the default path (/Users/au657961/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [9]:
# Figure saving options
flag_savefigs = False

fontsize_axis_ticks = 24
fontsize_axis_labels = 32

axis_thickness = 2

## Load the data

### Load the RRST behavioural data for the full sample

In [2]:
rrst_data_fpath = f'{data_dir}/rrst_behav_data_full.csv'
rrst_data_full = pd.read_csv(rrst_data_fpath)

# Rename selected columns
rrst_data_full = rrst_data_full.rename(columns={'ID': 'id'})

### Load the RRST behavioural data only for subject included in the VBQ analysis

In [3]:
vbq_data_fpath = f'{data_dir}/rrst_behav_data_vbq_subset.csv'
rrst_data_subset = pd.read_csv(vbq_data_fpath)

## Figures

### Group psychometric function

In [ ]:
fig_filename = 'rrst_group_pmf.png'

def cumGaussPsychFunc(x, intercept, slope, lower_asymptote):
    return lower_asymptote + (1 - lower_asymptote) * norm.cdf((x * slope + intercept))
    #return lower_asymptote + (1 - 2 * lower_asymptote) * norm.cdf((x - intercept) / slope)

x = np.linspace(2, 18, 1000)

# Create a square plot
plt.figure(figsize=(8.5,8.5))
mpl.rcParams['font.family'] = 'Helvetica'


# Add gridlines
plt.grid(True)
  
for index, row in rrst_data_full.iterrows():
    intercept = row['ind_esti_int']
    slope = row['ind_esti_slope']
    lower_asymptote = row['hier_guess']
    y = cumGaussPsychFunc(x, intercept, slope, lower_asymptote)
    # Draw grey functions with thinner lines
    plt.plot(x, y, color='lightgrey', linewidth=0.5)

scale_f = np.divide(100,17)
mean_intercept = rrst_data_full['ind_esti_int'].mean()
mean_slope = rrst_data_full['ind_esti_slope'].mean()
mean_lower_asymptote = rrst_data_full['hier_guess'].mean()
thresh_mean = rrst_data_full['real_intercept_ind'].mean()*scale_f
mean_slope_real = rrst_data_full['real_slope_ind'].mean()

thresh_sd = (np.std(rrst_data_full['real_intercept_ind'])) * scale_f
thresh_min = (np.min(rrst_data_full['real_intercept_ind'])) * scale_f
thresh_max = (np.max(rrst_data_full['real_intercept_ind'])) * scale_f

print('thresh_mean', thresh_mean, '\nthresh_sd', thresh_sd, '\nthresh_min', thresh_min, '\nthresh_max', thresh_max)
print('Slope mean: ',mean_slope_real)

y_mean = cumGaussPsychFunc(x, mean_intercept, mean_slope, mean_lower_asymptote)

# Draw the mean function with a thicker line
plt.plot(x, y_mean, color=(0.87, 0.76, 0.49), linewidth=8.0)

# Draw lines and point indicating mean threshold
mean_int_x = 12.5
plt.plot([np.min(x), mean_int_x], [0.75, 0.75], color='teal', linestyle='--', linewidth=4)
plt.plot([mean_int_x, mean_int_x], [np.min(y), 0.75], color='teal', linestyle='--', linewidth=4)
plt.scatter(12.5, 0.75, s=400, color='teal', edgecolor='teal', zorder=5) # s defines the size


plt.xlabel('% Obstruction', fontsize=fontsize_axis_labels)
plt.ylabel('p(correct response)', fontsize=fontsize_axis_labels)

plt.yticks(fontsize=fontsize_axis_ticks)
plt.xticks([3.4, 6.8, 10.2, 13.6, 17], ['20', '40', '60', '80', '100'], fontsize=fontsize_axis_ticks)

# Remove frame
ax = plt.gca()  # gca stands for 'get current axis'
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# Increase the thickness of the axis
for axis in ['bottom','left']:
    ax.spines[axis].set_linewidth(4)

# Add text for thresh value
plt.text(2.2, 0.95, 'Group threshold = 72.70%', fontsize=fontsize_axis_labels, bbox={'facecolor': 'teal', 'alpha': 0.25, 'pad': 8, 'edgecolor':'none'})

# Adjust layout to prevent clipping at the bottom
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust the bottom margin as needed (0.95 leaves 5% space at the bottom)


if flag_savefigs:
    # Save the figure with a specified size and resolution
    fig_filepath = f'{figures_dir}{fig_filename}'
    plt.savefig(fig_filepath, dpi=300)


plt.show()

